In [1]:
import urllib
import requests
import re
import pickle
from bs4 import BeautifulSoup

url = 'http://az.lib.ru/c/chehow_a_p/perepiska_tom1.shtml'
f = urllib.request.urlopen(url)
html = f.read()

soup = BeautifulSoup(html)

for script in soup(['script', 'style']):
    script.extract()

text = soup.get_text()
print(text[: 100])



Lib.ru/Классика: Чехов Антон Павлович. Переписка А. П. Чехова (Том первый)



Чехов Антон Павлович


**Итак, мы получили текст, начнем применять кустарные эвристики. В начале документа перечисляются письма, сгруппированные по адресатам, с датами и местом отправления. Потом эти данные будут дублироваться перед каждым письмом. Воспользуемся этим, чтобы узнать точное количество писем и список их заголовков. Для этого "вырежем" из текста кусок с перечислениями (он расположен между первым вхождением строк *start1* и *start2*).**

In [2]:
start1 = 'А. П. ЧЕХОВ И АЛ. П. ЧЕХОВ'
start2 = 'А. П. ЧЕХОВ В ПЕРЕПИСКЕ С СОВРЕМЕННИКАМИ'
letters_info = [string.strip() for string in text[text.find(start1): text.find(start2)].split('\n')]
letters_info = [string for string in letters_info if string != '' and not string.startswith('А. П. ЧЕХОВ ')]
print(len(letters_info))
for letter_info in letters_info:
    print(letter_info)

206
Ал. П. Чехов -- Чехову. 23 ноября 1877 г. Москва
Ал. П. Чехов -- Чехову. 14 октября 1878 г. Москва
Чехов -- Ал. П. Чехову. Март, не ранее 6, 1881 г. Москва
Чехов -- Ал. П. Чехову. 25 декабря 1882 г. и 1 или 2 января 1883 г. Москва
Чехов -- Ал. П. Чехову. 20-е числа февраля 1883 г. Москва
Чехов -- Ал. П. Чехову. 4 января 1886 г. Москва
Ал. П. Чехов -- Чехову. 17 января 1886 г. Новороссийск
Чехов -- Ал. П. Чехову. 3 февраля 1886 г. Москва
Чехов -- Ал. П. Чехову. 10 мая 1886 г. Москва
Ал. П. Чехов -- Чехову. 1 января 1887 г. Петербург
Чехов -- Ал. П. Чехову. 19 или 20 февраля 1887 г. Москва
Ал. П. Чехов -- Чехову. 14 июня 1887 г. Петербург
Чехов -- Ал. П. Чехову. 21 июня 1887 г. Бабкино
Чехов -- Ал. П. Чехову. Начало августа 1887 г. Бабкино
Ал. П. Чехов -- Чехову. 5--6 сентября 1887 г. Петербург
Чехов -- Ал. П. Чехову. 7 или 8 сентября 1887 г. Москва
Чехов -- Ал. П. Чехову. 10, 11 или 12 октября 1887 г. Москва
Ал. П. Чехов -- Чехову. 18 октября 1887 г. Петербург
Чехов -- Ал. П. Чехову

**Все получилось, теперь отрежем от каждой такой строки первую часть, описывающую отправителя и адресата. Заодно проверим, что в каждом такая нашлась.**

In [3]:
pattern = r'(Чехов -- [\. а-яА-ЯёЁ]*\. )|([\. а-яА-ЯёЁ]* -- Чехову. )'
dates_info = dict()
for letter_info in letters_info:
    match = re.search(pattern, letter_info)
    if match:
        dates_info[letter_info] = letter_info[len(match[0]):]
    else:
        print(letter_info)

Чехов -- A. Н. Плещееву. 19 января 1888 г. Москва


**Нашлась не в каждом, одну добавим руками.**

In [4]:
dates_info['Чехов -- A. Н. Плещееву. 19 января 1888 г. Москва'] = '19 января 1888 г. Москва'

**Сколько же у нас всего писем?**

In [5]:
print(len(dates_info))

206


**Это число нужно запомнить и стараться достичь именно его при парсинге. Далее начинаем напосредственно разбиение на письма в оставшейся части текста.**

In [6]:
cutted_text = text[text.find(start2):]

for date_info in dates_info.items():
    if cutted_text.find(date_info[1]) == -1:
        print(date_info)

('Чехов -- Н. А. Лейкину. 22 марта 1885 г. Москва', '22 марта 1885 г. Москва')
('Чехов -- Н. А. Лейкину. 25 января 1886 г. Москва', '25 января 1886 г. Москва')
('Чехов -- А. С. Суворину. 30 августа 1891 г. Вогимово', '30 августа 1891 г. Вогимово')
('Чехов -- А. С. Суворину. 27 март 1Я94 г. Ялта', '27 март 1Я94 г. Ялта')
('Чехов -- А. С. Суворину. 21 октября 1895 г. Мелихова', '21 октября 1895 г. Мелихова')
('Чехов -- А. С. Суворину. 6 (18) февраля 1898 г. Ницца', '6 (18) февраля 1898 г. Ницца')
('Чехов -- В. Г. Короленко. 2 мая 1888 г. Москва', '2 мая 1888 г. Москва')
('В. Г. Короленко -- Чехову. 4 августа 1902 г. Джанхот', '4 августа 1902 г. Джанхот')
('А. Н. Плещеев -- Чехову. 30 марта 1888 г. Петербург', '30 марта 1888 г. Петербург')
('А. Н. Плещеев -- Чехову. 5 ноября 1889 г. Петербург', '5 ноября 1889 г. Петербург')
('Чехов -- Я. П. Полонскому, 25 марта 1888 г. Москва', 'Полонскому, 25 марта 1888 г. Москва')


**Эти дато-временные описания почему-то в тексте не нашлись. Изучение глазами говорит нам об опечатках. Исправим руками на то, что в тексте найдется (это последний раз, когда мы что-то правим руками!).**

In [7]:
dates_info['Чехов -- Н. А. Лейкину. 22 марта 1885 г. Москва'] = '22 марта 1885 г. Моста'
dates_info['Чехов -- Н. А. Лейкину. 25 января 1886 г. Москва'] = '28 января 1886 г. Москва'
dates_info['Чехов -- А. С. Суворину. 30 августа 1891 г. Вогимово'] = '30 августа 1891 г. Богимово'
dates_info['Чехов -- А. С. Суворину. 27 март 1Я94 г. Ялта'] = '27 марта 1894 г. Ялта'
dates_info['Чехов -- А. С. Суворину. 21 октября 1895 г. Мелихова'] = '21 октября 1895 г. Мелихово'
dates_info['Чехов -- А. С. Суворину. 6 (18) февраля 1898 г. Ницца'] = '6(18) февраля 1898 г. Ницца'
dates_info['Чехов -- В. Г. Короленко. 2 мая 1888 г. Москва'] = '2 мая 1883 г. Москва'
dates_info['В. Г. Короленко -- Чехову. 4 августа 1902 г. Джанхот'] = '4 августа 1902 г, Джанхот'
dates_info['А. Н. Плещеев -- Чехову. 30 марта 1888 г. Петербург'] = '30 марта 188S г. Петербург'
dates_info['А. Н. Плещеев -- Чехову. 5 ноября 1889 г. Петербург'] = '5 ноября 1889 г, Петербург'
dates_info['Чехов -- Я. П. Полонскому, 25 марта 1888 г. Москва'] = '25 марта 1888 г. Москва'

In [8]:
for date_info in dates_info.items():
    if cutted_text.find(date_info[1]) == -1:
        print(date_info)

**Теперь все нашлось, хорошо. Найдем все моменты в тексте, претендующие, таким образом, на то, чтобы быть началом письма.**

In [9]:
starts = []
for date in dates_info.values():
    starts_date = [(match.start(), len(date)) for match in re.finditer(date.replace('(', '\(').replace(')', '\)'), cutted_text)]
    if len(starts_date) == 0:
        print(date)
    starts += starts_date
starts.sort()
maybe_letters = [cutted_text[starts[i][0] + starts[i][1]: starts[i + 1][0]] for i in range(len(starts) - 1)]
maybe_letters.append(cutted_text[starts[-1][0] + starts[-1][1]:])
print(len(maybe_letters))

221


**Тут несколько больше вхождений, чем нужно. Некоторые даты дублируются, иногда рукой отправителей писем (помимо автором сборника). Удалим лишнее.**

In [10]:
letters = [letter for letter in maybe_letters if len(letter) > 10]
print(len(letters))

206


**Получили наше магическое число. Сейчас в конце каждого письма заголовок следующего (от кого - кому), ведь мы резали по дате. Например:**

In [11]:
print(letters[0])


   


Москва. 23 ноября 1877 г.


   


Отъче Антоние!


   Сегодня я получил твою цыдулу1 и, придя из университета, сейчас же начал писать тебе сию грамоту. Когда я пошлю ее -- неизвестно, ибо марки нет. Сегодня же я должен послать велеречивое словоизвержение дядюшке Митрофану, ибо он именинник есть. Безденежье у меня теперь образцовое. Хотел было послать Митрофану Георгиевичу телеграмму, но рубля не оказалось. Все у нас благополучно. Маменька плачет не так часто. Росписание -- со стены исчезло2. Я по-прежнему живу отдельно от родных, но у них бываю довольно часто. Мишке в гимназии не везет, так что, я думаю, его после экзаменов попросят удалиться, ибо по латыни у него до сих пор отметка не заходит выше тройки, а двоек, а наипаче -- колов -- обилие. Николай мечтает о золотой медали и звании свободного художника и поэтому "начал" несколько картин. Я не совеем здоров, должно быть от недостаточного питания. Если пришлешь табаку, то это будет весьма кстати. У маменьки живет Наутилус3 и с

**Отрежем последнюю строчку и все пустые. Самое последнее письмо (у которого именно такой строчки нет) от этого явно не пострадает, там много лишнего в конце.**

In [12]:
letters = [letter.strip() for letter in letters]
for i in range(len(letters)):
    letters[i] = '\n'.join(letters[i].split('\n')[:-1]).strip()

**Удалим все, что добавлено составителями внизу письма: обычно там строка, описывающая происхожедния письма, в частности, диапазон страниц источника, номер тома или еще некоторые характерные штуки из регулярки ниже.**

In [13]:
pattern_pages = '(с\. (\d+(--|-)\d+))|(ГБЛ)|(Печатается по автографу)|(т. \d+, No \d+)'
for i in range(len(letters)):
    strings = letters[i].split('\n')
    min_bad_idx = len(strings)
    for idx, string in enumerate(strings):
        if re.search(pattern_pages, string):
            min_bad_idx = idx
            break
    strings = strings[: min_bad_idx]
    letters[i] = '\n'.join(strings).strip()

**В начале некоторых писем осталась дата, написанная отправителем. Удалим и ее, предположив, что это такая строка, в которой есть цифры и за которой есть пустая строка перед началом самого письма:**

In [14]:
for i in range(len(letters)):
    strings = letters[i].split('\n')
    if len(strings) >= 2 and len(re.findall('\d+', strings[0])) > 0 and strings[1] == '':
        letters[i] = '\n'.join(strings[1:]).strip()

**Сноски в тексте превратились в приклеенные цифры, перед которыми стоям буквы (и, возможно, кавычки или скобочки), а после - возможно, знак препинания.**

In [15]:
pattern = '[а-яА-ЯёЁ\(\)"]+\d+[\.,!?]*'

for i in range(len(letters)):
    words = letters[i].split()
    for j in range(len(words)):
        if re.match(pattern, words[j]):
            idx = 0
            while not words[j][idx].isdigit():
                idx += 1
            new_word = words[j][: idx]
            if not words[j][-1].isdigit():
                new_word += words[j][-1]
            letters[i] = letters[i].replace(words[j], new_word)

**Посмотрим на результат всего этого:**

In [16]:
for idx, letter in enumerate(letters):
    print(idx, '\n')
    print(letter)
    print('\n')

0 

Отъче Антоние!


   Сегодня я получил твою цыдулу и, придя из университета, сейчас же начал писать тебе сию грамоту. Когда я пошлю ее -- неизвестно, ибо марки нет. Сегодня же я должен послать велеречивое словоизвержение дядюшке Митрофану, ибо он именинник есть. Безденежье у меня теперь образцовое. Хотел было послать Митрофану Георгиевичу телеграмму, но рубля не оказалось. Все у нас благополучно. Маменька плачет не так часто. Росписание -- со стены исчезло. Я по-прежнему живу отдельно от родных, но у них бываю довольно часто. Мишке в гимназии не везет, так что, я думаю, его после экзаменов попросят удалиться, ибо по латыни у него до сих пор отметка не заходит выше тройки, а двоек, а наипаче -- колов -- обилие. Николай мечтает о золотой медали и звании свободного художника и поэтому "начал" несколько картин. Я не совеем здоров, должно быть от недостаточного питания. Если пришлешь табаку, то это будет весьма кстати. У маменьки живет Наутилус и с утра до ночи ссорится с ней, причем ник

**Бывает и лучше, но результат мы все-таки сохраним. Дальнейшая очистка будет происходить уже другими средствами.**

In [17]:
with open('chehow.pkl', 'wb') as f:
    pickle.dump(letters, f)